In [1]:
# import the required libraries as follows:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# loading the csv file
train = pd.read_csv('../input/train/train.csv')

 We have grayscale images, so while loading the images we will keep color_mode = "grayscale".
 
 Now, we will read all the training images, store them in a list, and finally convert that list into a numpy array.

In [3]:
# To store training images
train_image = []

for i in tqdm(range(train.shape[0])):
    img = image.load_img('../input/train/train/'+train['id'][i].astype('str')+'.png', target_size=(28,28,1), color_mode = "grayscale")
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 60000/60000 [02:27<00:00, 407.75it/s]


 The problem is a multi-class classification problem (10 classes).
 We will perform one-hot encoding on the target variable.

In [4]:
y=train['label'].values
y = to_categorical(y)

Create a validation set from the training data.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

Define the model structure.

We will create a simple architecture with 2 convolutional layers, one dense hidden layer and one output layer.

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [7]:
# compile the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [8]:
# train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 17s 346us/step - loss: 0.5124 - acc: 0.8171 - val_loss: 0.3174 - val_acc: 0.8807
Epoch 2/10
48000/48000 [==============================] - 10s 206us/step - loss: 0.3402 - acc: 0.8768 - val_loss: 0.2702 - val_acc: 0.8998
Epoch 3/10
48000/48000 [==============================] - 10s 206us/step - loss: 0.2920 - acc: 0.8942 - val_loss: 0.2460 - val_acc: 0.9070
Epoch 4/10
48000/48000 [==============================] - 10s 205us/step - loss: 0.2586 - acc: 0.9066 - val_loss: 0.2276 - val_acc: 0.9132
Epoch 5/10
48000/48000 [==============================] - 10s 205us/step - loss: 0.2320 - acc: 0.9155 - val_loss: 0.2330 - val_acc: 0.9132
Epoch 6/10
48000/48000 [==============================] - 10s 205us/step - loss: 0.2117 - acc: 0.9220 - val_loss: 0.2244 - val_acc: 0.9173
Epoch 7/10
48000/48000 [==============================] - 10s 205us/step - loss: 0.1934 - acc: 0.9272 - val_loss: 0.

In [9]:
# loading the csv file
test = pd.read_csv('../input/test/test.csv')

Now, we'll read all the test images, store them in a list, and then convert that list into a numpy array.

In [10]:
# creating an empty list
test_image = []
#
for i in tqdm(range(test.shape[0])):
    img = image.load_img('../input/test/test/'+test['id'][i].astype('str')+'.png', target_size=(28,28,1), color_mode = "grayscale")
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 10000/10000 [00:23<00:00, 434.01it/s]


Now, we'll make predictions. If you print "prediction", we'll see that the "prediction" is essentially an array containing all apparel labels.

In [11]:
# making predictions
prediction = model.predict_classes(test)
prediction

array([9, 2, 1, ..., 8, 1, 5])

As you've figured it out that the array obtained contains all predicted labels of the apparel images in test set.
Now, we convert the results into a csv file.

In [12]:
# creating submission file
sample = pd.read_csv('../input/test/test.csv')
sample['label'] = prediction
sample.to_csv('fashion_cnn.csv', header=True, index=False)